In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.metrics import roc_auc_score #精度評価に使用

/Users/takako/Desktop/GitHub/signate_cup_2024/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 各種定数の定義
# 乱数シード
RANDOM_SEED = 42
# target となる特徴量
TARGET_COULMN_NAME = "ProdTaken"
# 削除する特徴量のリスト
DROP_COLUMNS = []
# one-hot encoding する特徴量のリスト
ONE_HOT_ENCODING_COLUMNS = []
# ベストモデルのパス
BEST_MODEL_PATH = "../working/best_autogluon_model.pkl"

In [3]:
# 評価関数の定義
ag_roc_auc_scorer = make_scorer(
    name="roc_auc_score",
    score_func=roc_auc_score,
    optimum=1,
    greater_is_better=True,
    needs_threshold=True,
)

In [4]:
# 学習データの読み込み
train_df =  pd.read_csv("data/train.csv")
X_train_df = train_df.drop(columns=[TARGET_COULMN_NAME])

In [5]:
# テストデータの読み込み
test_df =  pd.read_csv("data/test.csv")
X_test_df = test_df

In [6]:
predictor = TabularPredictor(
        label=TARGET_COULMN_NAME,
        eval_metric=ag_roc_auc_scorer,
        path="../models",
    ).fit(
        train_df,
        # TODO: how to specify the random seed?
        # random_state=RANDOM_SEED,
        hyperparameters='default',
        hyperparameter_tune_kwargs={"search_strategy": "bayesopt"},
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.2
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:16:51 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T8103
CPU Count:          8
Memory Avail:       3.68 GB / 16.00 GB (23.0%)
Disk Space Avail:   55.95 GB / 926.35 GB (6.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for init

: 

In [ ]:
y_pred = predictor.predict_proba(X_test_df)